
#Introdução ao Apache Spark

Apache Spark é um framework de processamento de dados de código aberto projetado para ser rápido, fácil de usar e geral. Ele permite o processamento distribuído de grandes volumes de dados em clusters, suportando tarefas de processamento em lote e em tempo real. Spark é amplamente utilizado para análise de dados, aprendizado de máquina, processamento de streaming e muito mais.


#### História e Marcos do Apache Spark
*  2009: Desenvolvido originalmente na UC Berkeley’s AMPLab.
*  2010: Spark foi open-sourced.
*  2014: Tornou-se um projeto de alto nível da Apache Software Foundation.
*  2016: Spark 2.0 lançado, introduzindo DataFrames e Spark SQL.
*  2020: Spark 3.0 lançado, trazendo melhorias significativas em desempenho e novas APIs.
*  2024: Spark continua evoluindo, com foco em otimizações de desempenho e integração com tecnologias emergentes.

#Fundamentos do Apache Spark
#### RDDs (Resilient Distributed Datasets)

RDDs são a abstração fundamental no Spark, representando uma coleção distribuída de elementos que podem ser processados em paralelo. Eles são imutáveis e tolerantes a falhas.

##### Características:

* Imutabilidade: Uma vez criado, um RDD não pode ser alterado.
* Tolerância a falhas: Spark reconstrói automaticamente partes perdidas dos RDDs.
* Transformações e Ações: Operações que transformam os dados ou coletam resultados.

#### RDDs (Resilient Distributed Datasets)

#### **DataFrames**

DataFrames são uma abstração mais estruturada sobre RDDs, semelhantes a tabelas em bancos de dados relacionais. Eles permitem operações otimizadas e integração com Spark SQL.

**Vantagens:**

* Performance: Otimizações como o Catalyst Optimizer melhoram o desempenho.
* Facilidade de Uso: APIs mais amigáveis para manipulação de dados estruturados.
* Integração: Compatível com diversas fontes de dados e ferramentas de BI.


** Datasets Utilizados **

https://www.kaggle.com/datasets/prathamjyotsingh/apple-stocks



In [1]:
# Instalar PySpark via pip
!pip install pyspark

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("prathamjyotsingh/apple-stocks")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1


In [3]:
!ls -R /root/.cache/kagglehub/

/root/.cache/kagglehub/:
datasets

/root/.cache/kagglehub/datasets:
prathamjyotsingh

/root/.cache/kagglehub/datasets/prathamjyotsingh:
apple-stocks

/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks:
1.complete  versions

/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions:
1

/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1:
AAPL_stock_dividend.csv  AAPL_stock_price.csv  AAPL_stock_split.csv


In [4]:
# Iniciar uma Sessão Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploPySpark") \
    .getOrCreate()

**Comparando métodos do pandas x métodos do PySpark**

Leitura de Dados:

In [5]:
import pandas as pd
df_pandas = pd.read_csv("/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1/AAPL_stock_price.csv")

In [6]:
df_pandas.head(4)

,Date,Open Price,High Price,Low Price,Close Price,Volume
0,1999-11-01,80.00,80.69,77.37,77.62,2487300.0
1,1999-11-02,78.00,81.69,77.31,80.25,3564600.0
2,1999-11-03,81.62,83.25,81.00,81.50,2932700.0
3,1999-11-04,82.06,85.37,80.62,83.62,3384700.0


In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ExemploLeitura").getOrCreate()

# Leitura de um arquivo CSV
df_pyspark = spark.read.csv("/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1/AAPL_stock_price.csv", header=True, inferSchema=True)

In [8]:
#Nomes das colunas
for i in df_pandas.columns:
  print(i)

Date
Open Price
High Price
Low Price
Close Price
Volume


Seleção de Colunas:

In [9]:
# Seleciona colunas específicas - Pandas
df_selecionado_pandas = df_pandas[['Date', 'Open Price','High Price','Low Price','Volume']]

In [10]:
# Seleciona colunas específicas - PySpark
df_selecionado_spark = df_pyspark.select('Date', 'Open Price','High Price','Low Price','Volume')

Filtragem de Dados:

In [11]:
# Filtra linhas onde coluna1 > 50 - Pandas
df_filtrado_pandas = df_pandas[df_pandas['Volume'] > 2]

In [12]:
# Filtra linhas onde coluna1 > 50 - PySpark
df_filtrado_spark = df_pyspark.filter(df_pyspark.Volume > 2)

Agregações:

In [13]:
# Calcula a média de coluna1 agrupada por coluna2 - Pandas
df_agregado_pandas = df_pandas.groupby('Open Price')['High Price'].mean().reset_index()

In [14]:
# Calcula a média de coluna1 agrupada por coluna2 - PySpark
from pyspark.sql.functions import avg

# Calcula a média de coluna1 agrupada por coluna2
df_agregado_spark = df_pyspark.groupBy("Open Price").agg(avg("High Price").alias("media_days_left"))


In [15]:
df_selecionado_spark.show(5)
df_filtrado_spark.show(5)
df_agregado_spark.show(5)

+----------+----------+----------+---------+---------+
|      Date|Open Price|High Price|Low Price|   Volume|
+----------+----------+----------+---------+---------+
|1999-11-01|      80.0|     80.69|    77.37|2487300.0|
|1999-11-02|      78.0|     81.69|    77.31|3564600.0|
|1999-11-03|     81.62|     83.25|     81.0|2932700.0|
|1999-11-04|     82.06|     85.37|    80.62|3384700.0|
|1999-11-05|     84.62|     88.37|     84.0|3721500.0|
+----------+----------+----------+---------+---------+
only showing top 5 rows

+----------+----------+----------+---------+-----------+---------+
|      Date|Open Price|High Price|Low Price|Close Price|   Volume|
+----------+----------+----------+---------+-----------+---------+
|1999-11-01|      80.0|     80.69|    77.37|      77.62|2487300.0|
|1999-11-02|      78.0|     81.69|    77.31|      80.25|3564600.0|
|1999-11-03|     81.62|     83.25|     81.0|       81.5|2932700.0|
|1999-11-04|     82.06|     85.37|    80.62|      83.62|3384700.0|
|1999-11-05

Junções (Joins):

In [16]:
# Junção de dois DataFrames pandas
df1 = pd.read_csv("/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1/AAPL_stock_price.csv")
df2 = pd.read_csv("/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1/AAPL_stock_dividend.csv")
df_juncao = pd.merge(df1, df2, on="Date", how="left")

In [17]:
df1['Date'] = df1['Date'].astype(str)
df2['Date'] = df2['Date'].astype(str)

In [18]:
print(df1.dtypes)
print(df2.dtypes)

Date            object
Open Price     float64
High Price     float64
Low Price      float64
Close Price    float64
Volume         float64
dtype: object
Date                 object
Declaration Date     object
Record Date          object
Payment Date         object
Dividend            float64
dtype: object


In [19]:
# Junção de dois DataFrames PySpark
df1 = pd.read_csv("/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1/AAPL_stock_price.csv")
df2 = pd.read_csv("/root/.cache/kagglehub/datasets/prathamjyotsingh/apple-stocks/versions/1/AAPL_stock_dividend.csv")
df_juncao = df1.merge(df2, on="Date", how="left")

Criação de Novas Colunas:

In [20]:
# Cria uma nova coluna 'nova_coluna' como a soma de 'coluna1' e 'coluna2'
df_pandas['nova_coluna'] = df_pandas['Volume'] + df_pandas['Low Price']

In [21]:
from pyspark.sql.functions import col

# Cria uma nova coluna 'nova_coluna' como a soma de 'coluna1' e 'coluna2'
df_pyspark = df_pyspark.withColumn("nova_coluna", col("Volume") + col("Low Price"))


Ordenação de Dados:

In [22]:
# Ordena o DataFrame pelo 'coluna1' em ordem descendente
df_pandas_ordenado = df_pandas.sort_values(by='Date', ascending=False)

In [23]:
# Ordena o DataFrame pelo 'coluna1' em ordem descendente
df_pyspark_ordenado = df_pyspark.orderBy(df_pyspark.Date.desc())